<img width="50" src="https://carbonplan-assets.s3.amazonaws.com/monogram/dark-small.png" style="margin-left:0px;margin-top:20px"/>

# FIA Biomass Results

_by Jeremy Freeman (CarbonPlan), October 25, 2020_

This notebook extracts and summarizes results from fitting biomass growth curves. See the notebook `biomass_modeling` for how the model is fit. This notebook assumes the model has been run and results have been saved using the script `biomass.py`.


In [14]:
import pickle
import numpy as np
import pandas as pd
from carbonplan_forests import setup, plot, load

In [15]:
setup.plotting()

In [54]:
%load_ext autoreload
%autoreload 2

First we load the raw FIA and climate data for comparison purposes

In [45]:
df = load.fia(store="local", states="conus")
df = load.terraclim(
    store='local', tlim=(2000, 2020), data_vars=['tavg', 'ppt'], data_aggs=['mean', 'sum'], df=df
)

Now we load our model results. Again, this assumes the model has been run and results have been saved using the script `biomass.py`.

In [61]:
pf = pd.read_parquet('gs://carbonplan-scratch/forests/results/biomass/biomass.parquet')
# with open('../../scripts/data/biomass.pkl', 'rb') as f:
#     models = pickle.load(f)

First we can plot a map of biomass in the original data

In [48]:
plot.carto(
    lat=df['lat'],
    lon=df['lon'],
    color=df['biomass'],
    cmap='yellowgreen',
    clim=(0, 250),
    size=1,
)

alt.LayerChart(...)

Now we can plot the results from the model, evaluting on the same climate as used for fitting

In [49]:
plot.carto(
    lat=pf['lat'],
    lon=pf['lon'],
    color=pf['fitted_biomass'],
    cmap='yellowgreen',
    clim=(0, 250),
    size=1,
)

alt.LayerChart(...)

We can also use other columns to look at future scenarios. For example, here's the difference in biomass from 2020 to 2100.

In [13]:
plot.carto(
    lat=pf['lat'],
    lon=pf['lon'],
    color=pf['BCC-CSM2-MR_ssp245_2100'] - pf['BCC-CSM2-MR_ssp245_2020'],
    cmap='yellowgreen',
    clim=(0, 100),
    size=1,
)

alt.LayerChart(...)